In [16]:
from models import copy_model, model_DL2
from utils import rearrange_epochs
import numpy as np
import scipy.io as sio
from sklearn.metrics import roc_auc_score

In [3]:
# Step 1: Create a model and load weights from pre-training on Duke dataset. Adjust path in WEIGHTS_FILE variable
X = np.empty([100,12,256]) #batch size, channel number, samples number
Y = np.empty([100,1])
model=model_DL2(X,Y)
WEIGHTS_FILE='/media/ssd/projects/MLHC_results/models/DL6_Duke_Classbalance_FullDataset/PT130_DL6_Duke_FullDataset_class.model_weights'
model.load_weights(WEIGHTS_FILE)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# Step 2: Create a model suited for Helsinki dataset and copy weights to it
X_helsinki = np.empty([100,18,256])
Y_helsinki = np.empty([100,1])
model_helsinki = model_DL2(X_helsinki,Y_helsinki)
model_helsinki = copy_model(model,model_helsinki)

In [12]:
# Step 3: Load data of one subject from Helsinki dataset. 
# Extraction is done via adjusted code from Tapani et al. repository
# Seizure annotations are by consensus of 3 raters
matdata_fname = 'helsinki_eeg1_feats.mat'
annot_fname = 'helsinki_eeg1_cons_chan_annot.mat'
matdata = sio.loadmat(matdata_fname)
matannot = sio.loadmat(annot_fname)

In [13]:
# Step 4: Transform matdata and matannot into a suitable structure for prediction, and predict
# Duke dataset is trained on different scale of the data, so we need to add scaling coefficient sc

In [15]:
epochs, annot, _ = rearrange_epochs(matdata['all_feats'], matannot['annot'], 1,
                                                      None, n_chan=18)
sc = 0.001
epochs = epochs*sc
pred = model_helsinki.predict(epochs)

NaN epochs: 225
correct epochs: 1522


In [17]:
# Simple AUC score with annotations and predictions. Post-processing usually improves this AUC value.
roc_auc_score(annot,pred)

0.8993665648077414